# Домашнее задание 1 (до 27.09.2022)



1.   Сначала нам надо скачать дату -- соберите как минимум 60 (30 положительных и 30 отрицательных) отзывов на похожие продукты (не надо мешать отзывы на отели с отзывами на ноутбуки) для составления "тонального словаря" (чем больше отзывов, тем лучше) и 10 отзывов для проверки качества. (2 балла в случае сбора путём парсинга, 1 - если найдете уже готовые данные или просто закопипастите без парсинга)



# Мой план действий (ибо я совсем забыла, как делать это сходу):

1.  Будем парсить сайт Metacritic (`https://www.metacritic.com`).
2.  Возьмем фильмы жанра Fantasy(`https://www.metacritic.com/browse/movies/genre/userscore/fantasy?view=detailed`), отсортированные по пользовательскому баллу.
3. Посмотрим в HTML-раскладке кол-во страниц, перейдем на последнюю (строка `<ul class="pages">`)
4. С последней страницы возьмем 20 (может потом поменяю) последних фильмов
5. Сохраним списком ссылки на их рецензии
6. Проходясь по спискам, отсортированным по полезности (`
?sort-by=most-helpful&num_items=100`), будем брать первые 5 отзывов
7. Определяем статус (+/-) фильма:
Нам нужны либо
*   Отличные `<div class="metascore_w user large movie positive indiv perfect">`(10)
*  Хорошие `<div class="metascore_w user large movie positive indiv">` (9-7)
*  Отстойные `<div class="metascore_w user large movie negative indiv">` (3-0)
8. Сохраняем текст отзыва в соответствующий документ (все равно, потом все обрабатывать): `<div class="review_body">`
9. То же (4-8) делаем с фильмами с первой страницы
10. Глазками находим (с других страниц) 5 положительных и 5 отрицательных отзывов
На выход: два документа (+/-) пробных, два документа отзывов (+/-) 


In [3]:
!pip install fake_useragent

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=fb91447779b63eb84de052e5ac768e286d7c2a836351f2cf1ff1a21d7bbff162
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
!pip install langid

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
import requests
from pprint import pprint
import sqlite3
from bs4 import BeautifulSoup
import re
from fake_useragent import UserAgent
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('wordnet')
import langid
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
sw = stopwords.words('english')
m = WordNetLemmatizer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [29]:
from fake_useragent import UserAgent
fake_me = UserAgent(verify_ssl=False)
session = requests.session()

In [30]:
def parser(url):
  req = session.get(url, headers={'User-Agent': fake_me.random})
  page = req.text
  soup = BeautifulSoup(page, 'html.parser')
  return soup

In [31]:
url = f'https://www.metacritic.com/browse/movies/genre/userscore/fantasy?view=detailed.html'
soup = parser(url)
page_num = soup.find_all('ul', {'class': 'pages'})
pages_links = re.findall(r'/browse/movies/genre/userscore/fantasy\?view=detailed\.html&amp;page=\d\d*', str(page_num)) 
# Теперь у нас есть ссылки на каждую из страниц, хотя мы возьмем только первую и последнюю 

url_page = 'http://www.metacritic.com/browse/movies/genre/userscore/fantasy?view=detailed.html&%3Bpage=11&page=' + str(len(pages_links)+1)
soup = parser(url_page)
last_page = soup.find_all('table', {'clamp-list'})

reviews = re.findall(r'/movie/\w+[-\w]*/user-reviews', str(last_page))
rev_links = []
for i in range(len(reviews)-1):
  if i%2 == 0 and i >= len(reviews)-61:
    rev_links.append(reviews[i]) # список ссылок 30 фильмов 
len(rev_links)

30

In [32]:
def the_texts(texts): # Просто обрабатываем текст отзыва от HTMLевской шолухи
  rev_text = texts.find_all('div', {'class': "review_body"}) 
  rev_text = re.sub('\[<.*>', '', str(rev_text))
  rev_text = re.sub('<span>', '', str(rev_text))
  rev_text = re.sub('<.*>', '', str(rev_text))
  rev_text = re.sub('\]', '', str(rev_text))
  return rev_text

In [36]:
with open('Bad reviews.txt', 'a') as bad_file:
  with open('Good reviews.txt', 'a') as good_file:
    for film in rev_links:
      url_film = 'https://www.metacritic.com' + str(film) + '?sort-by=most-helpful&num_items=100'
      soup = parser(url_film)
      pad = soup.find_all('div', {'class': "review pad_top1"})
      bad = 0
      good = 0
      for i in range(len(pad)-1):  # Отзывы на один фильм
        rev_status = pad[i].find_all('div', {'class': "metascore_w user large movie negative indiv"})
        if len(rev_status) == 1: # отзыв негативный
          if "      This review contains spoilers." not in str(the_texts(pad[i])) and len(str(the_texts(pad[i]))) > 10:
            if langid.classify(str(the_texts(pad[i])))[0] =='en':
              bad += 1
              if bad <=5:
                bad_file.write(the_texts(pad[i]))
        else:
          rev_status = pad[i].find_all('div', {'class': "metascore_w user large movie positive indiv perfect"})
          rev_status_2 = pad[i].find_all('div', {'class': "metascore_w user large movie positive indiv"})
          if len(rev_status) == 1 or len(rev_status_2) == 1: # отзыв позитивный
            if "      This review contains spoilers." not in str(the_texts(pad[i])) and len(str(the_texts(pad[i]))) > 10:
              if langid.classify(str(the_texts(pad[i])))[0] =='en':
                good += 1
                if good <=5:
                  good_file.write(the_texts(pad[i]))

2.   Токенизируйте слова, приведите их к нижнему регистру и к начальной форме (1 балл за токенизацию, 1 - за начальную форму)



# План действий:


1.   Имеем на входе два документа
2.   Сделаем общую ф-цию для лемматизации и токенизации



In [37]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
def tokNlem(text):
  lems = []
  tocs = [w.lower() for w in word_tokenize(text) if w.isalpha()]
  filtered = [w for w in tocs if w not in sw]
  for word in filtered:
    lems.append(m.lemmatize(word))
  return lems


In [39]:
lem_bad = []
lem_good = []
with open('Bad reviews.txt') as bad_file:
  bad_text = bad_file.read()
  bad_lems = tokNlem (bad_text)
print(len(bad_lems), bad_lems)

with open('Good reviews.txt') as good_file:
  good_text = good_file.read()
  good_lems = tokNlem (good_text)
print(len(good_lems), good_lems)

1236 ['positive', 'nick', 'fury', 'great', 'hope', 'one', 'day', 'get', 'proper', 'shield', 'movie', 'goose', 'cat', 'good', 'boooring', 'lead', 'actor', 'brie', 'larson', 'terrible', 'uncharismatic', 'actually', 'first', 'female', 'superhero', 'marvel', 'would', 'prefer', 'never', 'see', 'watched', 'film', 'morning', 'really', 'regret', 'watching', 'theatre', 'let', 'start', 'good', 'thing', 'first', 'opening', 'action', 'scene', 'really', 'good', 'nick', 'fury', 'good', 'samuel', 'jackson', 'movie', 'nothing', 'good', 'offer', 'sad', 'endgame', 'far', 'one', 'boring', 'marvel', 'movie', 'really', 'bad', 'performance', 'brie', 'larson', 'without', 'emotion', 'feminist', 'propaganda', 'make', 'movie', 'one', 'worst', 'marvel', 'movie', 'ever', 'critic', 'really', 'missed', 'mark', 'one', 'mushu', 'song', 'watered', 'boring', 'mulan', 'sprinkled', 'familiar', 'superhero', 'trope', 'completely', 'joyless', 'remake', 'song', 'character', 'development', 'improvement', 'original', 'save', '



3.   Составьте 2 множества - в одном будут слова, которые встречаются только в положительных отзывах, а в другом - встречающиеся только в отрицательных. Попробуйте поиграть с частотностями и исключить шум (к примеру, выбросить слова, встречающиеся 1-2 раза) (2 балла) (если у вас получились пустые множества, уберите фильтр по частотности или увеличьте выборку)





# План действий:


1. Идем по словам одного списка и ищем каждое (find-ом) в другом, если его нет, добавляем слово в словарь, если оно там уже есть, увеличиваем на единицу значение
2. То же самое делаем в обратную сторону



In [40]:
good_dct = {}
bad_dct = {}
for word in bad_lems:
  if word not in good_lems:
    if word not in bad_dct:
      bad_dct[word] = 1
    else:
      bad_dct[word] = bad_dct[word] + 1

for word in good_lems:
  if word not in bad_lems:
    if word not in good_dct:
      good_dct[word] = 1
    else:
      good_dct[word] = good_dct[word] + 1
  
print(good_dct)
print(bad_dct)




{'family': 2, 'girlfriend': 1, 'loved': 3, 'experience': 1, 'photo': 1, 'soundtrack': 1, 'balanced': 1, 'beautiful': 1, 'nerdy': 2, 'completly': 1, 'war': 1, 'go': 2, 'seriously': 2, 'best': 6, 'upside': 1, 'lollypop': 1, 'nose': 1, 'sane': 1, 'extremely': 1, 'amazing': 4, 'pretty': 2, 'corny': 1, 'dialogue': 1, 'plenty': 1, 'giant': 1, 'monster': 1, 'sure': 3, 'might': 2, 'storyline': 1, 'pro': 1, 'overwhelmingly': 1, 'surpass': 1, 'con': 1, 'disrepec': 1, 'fav': 1, 'phat': 1, 'boiiii': 1, 'desrespec': 1, 'milk': 1, 'visually': 2, 'stunning': 2, 'follows': 1, 'different': 1, 'format': 1, 'typical': 1, 'diverse': 1, 'child': 2, 'adult': 1, 'alike': 1, 'fresh': 1, 'narrative': 1, 'inspiring': 2, 'gentle': 1, 'wrinkle': 1, 'sweet': 1, 'sentimental': 1, 'emotionally': 1, 'enchanting': 1, 'appealing': 1, 'goodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgoodgood': 1, 'catwoman': 3, 'adrenaline': 1, 'favourite': 1, 'version': 2, 'dc': 1, 'understand': 2, 'everyone': 3

4. Создайте функцию, которая будет определять, положительный ли отзыв или отрицательный в зависимости от того, какие слова встретились в нём, и посчитайте качество при помощи accuracy (1 - за коректно работающую функцию, 1 - за подсчёт accuracy)

# План действий:


1.   Токенизируем входной файл, приводим слова к начальной форме
2.   Идем по словам в получившемся списке и проверяем их наличие в двух списках


*   Если слово есть в одном из списков, добавляем что-то к общему score (Можно прибавлять в случае, если слово из + списка, и вычитать, если из -)
*   




In [75]:

def pos_neg(dct, lst):
  score = 0
  for word in lst:
    if word in dct:
      if dct[word] < 3:
        score += 1
      elif dct[word] <5 and dct[word] > 2:
        score += 2 
      elif dct[word] > 4:
        score += 3 
  return score 

In [44]:
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=3012137bba99312ca6c141a412883b8d9416ed0fde50848578f83481b3dd8100
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


In [81]:
from sklearn.metrics import accuracy_score
gold = ['pos', 'pos', 'pos', 'pos', 'pos', 'neg', 'neg', 'neg', 'neg', 'neg']
result = ''
results = []

In [82]:
with open ('Good examples.txt') as file:
  for line in file:
    good_exm = line
    gex_lems = tokNlem (good_exm)
    pos_score = pos_neg(good_dct, gex_lems)
    neg_score = pos_neg(bad_dct, gex_lems)
    if pos_score > neg_score:
      result = 'pos'
    if pos_score < neg_score:
      result = 'neg'
    else:
      result = 'neu'
    print(result, good_exm)
    if result == 'neu' or result == 'neg' or result == 'pos':
      print(result, good_exm)
      results.append(result)
with open ('Bad examples.txt') as file:
  for line in file:
    bad_exm = line
    bex_lems = tokNlem (bad_exm)
    pos_score = pos_neg(good_dct, bex_lems)
    neg_score = pos_neg(bad_dct, bex_lems)
    if pos_score > neg_score:
      result = 'pos'
    if pos_score < neg_score:
      result = 'neg'
    else:
      result = 'neu'
    if result == 'neu' or result == 'neg' or result == 'pos':
      print(result, bad_exm)
      results.append(result)
    
print("Accuracy: %.4f" % accuracy_score(results, gold))



neg It's surprising when you find one of these rare epics that still manage to captivate and spellbind over such a long period of time. The visuals were absolutely majestic, yet the story managed to unfold with such grace and never dawdled. The actors proved worthy of the role they played and kept up their spirit throughout. It was also nice to feel a little light-heartedness among the fantasy, drama and suspense that occurred. To be honest, I couldn't really point out what was actually bad about it, because it just looks like one of the best fantasy epics ever made.

neg It's surprising when you find one of these rare epics that still manage to captivate and spellbind over such a long period of time. The visuals were absolutely majestic, yet the story managed to unfold with such grace and never dawdled. The actors proved worthy of the role they played and kept up their spirit throughout. It was also nice to feel a little light-heartedness among the fantasy, drama and suspense that occ

# Общий комментарий:
Результат не очень хороший (точность 0,5), полагаю за счет того, что в выборке пошел перекос: слов в негативной подборке в 2 раза больше, чем в позитивной, если это исправить, думаю, все станет намного лучше (но у меня уже нет на это времени, т.к. надо срочно отправить дз.)

5. Предложите как минимум 2 способа улучшить эту программу с помощью добавления к ней любых мулек (1 балл за описание словами, 2 - если реализуете хотя бы один способ)

# Идея улучшения:


*   Исходя из нашего частотного словаря, принимать во внимание, что одни слова чаще встречаются в (положительных/ отрицательных) отзывах,чем другие. (+)
* Выкинуть все стоп-слова (они, вероятно и так выкинуться, но это сократит время работы алгоритма) (+)
* Лемматизировать слова (+)
* Проверять язык отзывов в тестовой выборке, чтобы иноязычный мусор не портил статистику(+)
*   Учитывать контекст (какие-то ближайшие соседние слова)
*   Учитывать специфику отзыва (на что он сделан.) Например для стройматериала жесткий - это хорошо, а для дивана - нет



